# ML Exploration: Days 2-3 - Feature Engineering & Baseline Model

This notebook covers:

**Day 2 - Feature Engineering Prep:**
- Feature extraction planning
- Train/test split design
- Helper similarity functions

**Day 3 - Baseline Model Construction:**
- Weighted popularity scoring
- Global movie ranking
- Top-N recommendations

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.spatial.distance import jaccard

# Import our feature engineering module
import sys
sys.path.append('../src')
from feature_engineering import (
    encode_genres_onehot,
    vectorize_tags_tfidf,
    create_year_features,
    FeaturePipeline
)

# Import baseline model functions
from baseline_model import (
    compute_weighted_popularity_score,
    compute_popularity_scores,
    rank_movies_globally,
    filter_by_minimum_votes,
    get_top_n_recommendations,
    get_personalized_top_n,
    compute_rating_statistics,
    merge_movie_with_stats
)

---

# Part 1: Feature Extraction Planning

## Overview

For our movie recommendation system, we will extract several types of features:

### 1.1 Genre Features (Categorical → Binary)
- **Source**: `genres` column (e.g., "Action|Comedy|Drama")
- **Encoding**: One-hot (multi-label) encoding
- **Rationale**: Genres are the primary content descriptor and form the basis of content-based filtering

### 1.2 Tag Features (Text → Numeric)
- **Source**: User-generated tags
- **Encoding**: TF-IDF vectorization
- **Rationale**: Tags capture nuanced aspects of movies that genres miss (e.g., "twist ending", "dark humor")

### 1.3 Temporal Features
- **Source**: Year extracted from movie title (e.g., "Toy Story (1995)")
- **Derived Features**:
  - Release year (numeric)
  - Decade (categorical/numeric)
- **Rationale**: User preferences often correlate with movie eras

## 1.4 Feature Extraction Strategy

```
Raw Data → Preprocessing → Feature Extraction → Feature Matrix

movies.csv ─────┬───> Genre One-Hot ────────────┐
                │                               │
tags.csv ───────┴───> Tag TF-IDF ───────────────┼───> Combined Feature Matrix
                │                               │
movie titles ───┴───> Year/Decade Extraction ───┘
```

In [ ]:
# Example: Feature extraction plan demonstration

# Sample data for demonstration
sample_movies = pd.DataFrame({
    'movieId': [1, 2, 3],
    'title': ['Toy Story (1995)', 'Jumanji (1995)', 'Heat (1995)'],
    'genres': ['Animation|Children|Comedy', 'Adventure|Children|Fantasy', 'Action|Crime|Thriller'],
    'tags': ['pixar animated fun', 'jungle adventure board game', 'heist robbery deniro pacino']
})

print("Sample Movies Data:")
print(sample_movies)
print("\n" + "="*60)

In [ ]:
# Demonstrate genre one-hot encoding
genre_features, genre_encoder = encode_genres_onehot(sample_movies, 'genres')
print("Genre One-Hot Encoding:")
print(genre_features)
print(f"\nGenre classes: {genre_encoder.classes_}")

In [ ]:
# Demonstrate year extraction
year_features = create_year_features(sample_movies, 'title')
print("Year Features:")
print(year_features)

---

# Part 2: Train/Test Split Design

## 2.1 Split Strategy Considerations

For recommendation systems, we must carefully consider how to split data:

### Random Split
- **Use Case**: Standard evaluation of model performance
- **Pros**: Simple, unbiased sample
- **Cons**: May leak temporal patterns

### Temporal Split
- **Use Case**: Simulating real-world deployment
- **Pros**: More realistic evaluation
- **Cons**: May bias toward recent items

### User-based Split
- **Use Case**: Evaluating cold-start scenarios
- **Pros**: Tests generalization to new users
- **Cons**: Different user behavior patterns

## 2.2 Split Configuration

In [ ]:
# Train/Test Split Configuration

SPLIT_CONFIG = {
    'test_size': 0.2,           # 20% for testing
    'validation_size': 0.1,     # 10% for validation (from training)
    'random_state': 42,         # For reproducibility
    'stratify': True,           # Stratify by rating distribution
}

print("Split Configuration:")
for key, value in SPLIT_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
def create_train_val_test_split(
    df: pd.DataFrame,
    test_size: float = 0.2,
    val_size: float = 0.1,
    random_state: int = 42,
    stratify_column: str = None
):
    """
    Create train/validation/test splits for the dataset.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe to split
    test_size : float
        Proportion of data for test set
    val_size : float
        Proportion of training data for validation set
    random_state : int
        Random seed for reproducibility
    stratify_column : str, optional
        Column to use for stratified splitting
        
    Returns:
    --------
    train_df, val_df, test_df : tuple of DataFrames
    """
    stratify = df[stratify_column] if stratify_column and stratify_column in df.columns else None
    
    # First split: separate test set
    train_val_df, test_df = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify
    )
    
    # Second split: separate validation from training
    stratify_val = train_val_df[stratify_column] if stratify_column and stratify_column in df.columns else None
    
    # Adjust validation size relative to remaining data
    val_size_adjusted = val_size / (1 - test_size)
    
    train_df, val_df = train_test_split(
        train_val_df,
        test_size=val_size_adjusted,
        random_state=random_state,
        stratify=stratify_val
    )
    
    return train_df, val_df, test_df

In [ ]:
def create_temporal_split(
    df: pd.DataFrame,
    timestamp_column: str,
    test_ratio: float = 0.2,
    val_ratio: float = 0.1
):
    """
    Create train/validation/test splits based on timestamp.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe with timestamp column
    timestamp_column : str
        Name of the timestamp column
    test_ratio : float
        Proportion of most recent data for test set
    val_ratio : float
        Proportion of data for validation set
        
    Returns:
    --------
    train_df, val_df, test_df : tuple of DataFrames
    """
    df_sorted = df.sort_values(timestamp_column)
    n = len(df_sorted)
    
    test_start_idx = int(n * (1 - test_ratio))
    val_start_idx = int(n * (1 - test_ratio - val_ratio))
    
    train_df = df_sorted.iloc[:val_start_idx]
    val_df = df_sorted.iloc[val_start_idx:test_start_idx]
    test_df = df_sorted.iloc[test_start_idx:]
    
    return train_df, val_df, test_df

In [ ]:
# Demonstrate splits with sample data

sample_ratings = pd.DataFrame({
    'userId': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4],
    'movieId': [1, 2, 3, 1, 2, 4, 2, 3, 4, 1],
    'rating': [4.0, 3.5, 5.0, 4.5, 3.0, 4.0, 5.0, 4.5, 3.5, 4.0],
    'timestamp': [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009]
})

print("Sample Ratings Data:")
print(sample_ratings)
print("\n" + "="*60)

# Random split
train, val, test = create_train_val_test_split(sample_ratings, test_size=0.2, val_size=0.1)
print(f"\nRandom Split Sizes: Train={len(train)}, Val={len(val)}, Test={len(test)}")

---

# Part 3: Helper Similarity Functions

## 3.1 Similarity Metrics Overview

For content-based filtering, we need to compute similarity between items:

| Metric | Best For | Range |
|--------|----------|-------|
| Cosine | Sparse vectors (TF-IDF) | [-1, 1] |
| Jaccard | Binary features (genres) | [0, 1] |
| Euclidean | Dense numeric features | [0, ∞) |
| Pearson | Rating patterns | [-1, 1] |

In [ ]:
def compute_cosine_similarity(feature_matrix: np.ndarray) -> np.ndarray:
    """
    Compute pairwise cosine similarity between all items.
    
    Parameters:
    -----------
    feature_matrix : np.ndarray
        Matrix of shape (n_items, n_features)
        
    Returns:
    --------
    similarity_matrix : np.ndarray
        Matrix of shape (n_items, n_items) with pairwise similarities
    """
    return cosine_similarity(feature_matrix)

In [ ]:
def compute_jaccard_similarity(binary_matrix: np.ndarray) -> np.ndarray:
    """
    Compute pairwise Jaccard similarity for binary features.
    
    Parameters:
    -----------
    binary_matrix : np.ndarray
        Binary matrix of shape (n_items, n_features)
        
    Returns:
    --------
    similarity_matrix : np.ndarray
        Matrix of shape (n_items, n_items) with pairwise Jaccard similarities
    """
    n_items = binary_matrix.shape[0]
    similarity_matrix = np.zeros((n_items, n_items))
    
    for i in range(n_items):
        for j in range(i, n_items):
            # Jaccard = intersection / union
            intersection = np.sum(np.logical_and(binary_matrix[i], binary_matrix[j]))
            union = np.sum(np.logical_or(binary_matrix[i], binary_matrix[j]))
            
            if union == 0:
                sim = 0.0
            else:
                sim = intersection / union
            
            similarity_matrix[i, j] = sim
            similarity_matrix[j, i] = sim
    
    return similarity_matrix

In [ ]:
def compute_euclidean_similarity(
    feature_matrix: np.ndarray,
    normalize: bool = True
) -> np.ndarray:
    """
    Compute similarity based on Euclidean distance.
    
    Parameters:
    -----------
    feature_matrix : np.ndarray
        Matrix of shape (n_items, n_features)
    normalize : bool
        If True, normalize distances to [0, 1] range and convert to similarity
        
    Returns:
    --------
    similarity_matrix : np.ndarray
        Matrix of shape (n_items, n_items) with pairwise similarities
    """
    distances = euclidean_distances(feature_matrix)
    
    if normalize:
        # Convert distance to similarity: sim = 1 / (1 + distance)
        similarity_matrix = 1 / (1 + distances)
    else:
        similarity_matrix = -distances  # Negative distance as similarity
    
    return similarity_matrix

In [ ]:
def get_top_similar_items(
    similarity_matrix: np.ndarray,
    item_idx: int,
    top_n: int = 10,
    exclude_self: bool = True
) -> list:
    """
    Get the top N most similar items to a given item.
    
    Parameters:
    -----------
    similarity_matrix : np.ndarray
        Precomputed similarity matrix
    item_idx : int
        Index of the query item
    top_n : int
        Number of similar items to return
    exclude_self : bool
        Whether to exclude the item itself from results
        
    Returns:
    --------
    similar_items : list of tuples
        List of (item_idx, similarity_score) tuples
    """
    similarities = similarity_matrix[item_idx]
    
    if exclude_self:
        # Set self-similarity to -inf to exclude
        similarities = similarities.copy()
        similarities[item_idx] = -np.inf
    
    # Get indices of top N similar items
    top_indices = np.argsort(similarities)[::-1][:top_n]
    
    return [(idx, similarities[idx]) for idx in top_indices]

In [ ]:
def compute_weighted_similarity(
    feature_matrices: list,
    weights: list,
    similarity_fn=cosine_similarity
) -> np.ndarray:
    """
    Compute weighted combination of multiple similarity matrices.
    
    Parameters:
    -----------
    feature_matrices : list of np.ndarray
        List of feature matrices (one per feature type)
    weights : list of float
        Weights for each feature type (should sum to 1)
    similarity_fn : callable
        Similarity function to use
        
    Returns:
    --------
    combined_similarity : np.ndarray
        Weighted combination of similarity matrices
    """
    if len(feature_matrices) != len(weights):
        raise ValueError("Number of matrices must match number of weights")
    
    # Normalize weights
    weights = np.array(weights) / np.sum(weights)
    
    combined_similarity = None
    
    for matrix, weight in zip(feature_matrices, weights):
        sim_matrix = similarity_fn(matrix)
        
        if combined_similarity is None:
            combined_similarity = weight * sim_matrix
        else:
            combined_similarity += weight * sim_matrix
    
    return combined_similarity

In [ ]:
# Demonstrate similarity computations

# Use genre features from earlier
print("Genre Features Matrix:")
print(genre_features.values)
print("\n" + "="*60)

# Compute cosine similarity
cosine_sim = compute_cosine_similarity(genre_features.values)
print("\nCosine Similarity Matrix:")
print(np.round(cosine_sim, 3))

# Compute Jaccard similarity
jaccard_sim = compute_jaccard_similarity(genre_features.values)
print("\nJaccard Similarity Matrix:")
print(np.round(jaccard_sim, 3))

In [ ]:
# Find most similar movies to Toy Story (index 0)
print("Movies most similar to 'Toy Story':")
similar_to_toy_story = get_top_similar_items(cosine_sim, item_idx=0, top_n=2)

for idx, score in similar_to_toy_story:
    print(f"  {sample_movies.iloc[idx]['title']}: similarity = {score:.3f}")

---

# Part 4: Baseline Model Construction (Day 3)

## 4.1 Weighted Popularity Scoring

The baseline model uses a **weighted popularity score** inspired by IMDB's weighted rating formula:

$$\text{Weighted Score} = \frac{v}{v + m} \cdot R + \frac{m}{v + m} \cdot C$$

Where:
- $v$ = number of votes (ratings) for the movie
- $m$ = minimum votes required (threshold)
- $R$ = average rating of the movie
- $C$ = global mean rating across all movies

This approach:
- Penalizes movies with few ratings (prevents outliers)
- Rewards movies with many ratings (more reliable)
- Provides a smooth transition between low and high vote counts

In [ ]:
# Example: Demonstrate weighted popularity scoring

# Sample movie data with rating statistics
sample_movies_with_ratings = pd.DataFrame({
    'movieId': [1, 2, 3, 4, 5],
    'title': ['Movie A (High Rated, Many Reviews)', 
              'Movie B (High Rated, Few Reviews)',
              'Movie C (Average Rated, Many Reviews)',
              'Movie D (Low Rated, Few Reviews)',
              'Movie E (Very High Rated, Few Reviews)'],
    'genres': ['Action|Drama', 'Comedy', 'Action|Thriller', 'Horror', 'Drama|Romance'],
    'avg_rating': [4.5, 4.8, 3.5, 2.0, 5.0],
    'rating_count': [500, 5, 300, 10, 3]
})

print("Sample Movies with Rating Statistics:")
print(sample_movies_with_ratings[['title', 'avg_rating', 'rating_count']])
print("\n" + "="*60)

In [ ]:
# Calculate global mean
global_mean = sample_movies_with_ratings['avg_rating'].mean()
print(f"Global Mean Rating: {global_mean:.2f}")
print("\nComparing individual scores:")

for idx, row in sample_movies_with_ratings.iterrows():
    score = compute_weighted_popularity_score(
        avg_rating=row['avg_rating'],
        rating_count=row['rating_count'],
        global_mean=global_mean,
        min_votes=10
    )
    print(f"  {row['title'][:40]:40s} | Avg: {row['avg_rating']:.1f} | Count: {row['rating_count']:3d} | Score: {score:.3f}")

In [ ]:
# Batch compute popularity scores for all movies
scores = compute_popularity_scores(
    sample_movies_with_ratings,
    rating_column='avg_rating',
    count_column='rating_count',
    min_votes=10
)

print("Computed Popularity Scores:")
print(scores)

## 4.2 Global Movie Ranking

Once we have popularity scores, we can rank movies globally to identify the most popular ones.

In [ ]:
# Rank movies globally by popularity score
ranked_movies = rank_movies_globally(
    sample_movies_with_ratings,
    rating_column='avg_rating',
    count_column='rating_count',
    min_votes=10
)

print("Movies Ranked by Weighted Popularity Score:")
print(ranked_movies[['title', 'avg_rating', 'rating_count', 'popularity_score']])

In [ ]:
# Filter movies by minimum vote threshold
filtered_movies = filter_by_minimum_votes(
    sample_movies_with_ratings,
    count_column='rating_count',
    min_votes=10
)

print(f"Movies with >= 10 votes ({len(filtered_movies)} movies):")
print(filtered_movies[['title', 'rating_count']])

## 4.3 Top-N Recommendations

The baseline model provides two types of recommendations:

1. **Global Top-N**: Best movies overall (non-personalized)
2. **Personalized Top-N**: Best movies excluding those already watched

In [ ]:
# Create a larger sample dataset for demonstration
larger_sample = pd.DataFrame({
    'movieId': range(1, 11),
    'title': [
        'The Shawshank Redemption (1994)', 'The Godfather (1972)',
        'The Dark Knight (2008)', 'Pulp Fiction (1994)',
        'Fight Club (1999)', 'Inception (2010)',
        'The Matrix (1999)', 'Goodfellas (1990)',
        'The Silence of the Lambs (1991)', 'Interstellar (2014)'
    ],
    'avg_rating': [4.9, 4.8, 4.7, 4.6, 4.5, 4.6, 4.5, 4.4, 4.3, 4.5],
    'rating_count': [250, 180, 350, 200, 150, 300, 400, 100, 120, 280]
})

print("Larger Sample Dataset:")
print(larger_sample)

In [ ]:
# Get global top-5 recommendations
top_5 = get_top_n_recommendations(
    larger_sample,
    n=5,
    min_votes=50
)

print("Top 5 Movie Recommendations (Global):")
print(top_5[['title', 'popularity_score']])

In [ ]:
# Personalized recommendations - exclude movies user has already watched
user_watched_movies = [1, 3, 7]  # User has watched movies with these IDs

personalized_recs = get_personalized_top_n(
    larger_sample,
    user_watched_ids=user_watched_movies,
    n=5,
    min_votes=50
)

print(f"Personalized Top 5 (excluding watched movies: {user_watched_movies}):")
print(personalized_recs[['movieId', 'title', 'popularity_score']])

## 4.4 Working with Real Rating Data

In practice, we start with raw ratings and compute statistics before applying the baseline model.

In [ ]:
# Simulate raw ratings data
raw_ratings = pd.DataFrame({
    'userId': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5,
               6, 6, 7, 7, 7, 8, 8, 9, 9, 10, 10, 10, 10],
    'movieId': [1, 2, 3, 1, 2, 4, 1, 3, 5, 2, 3, 5, 1, 4, 5,
                1, 2, 1, 3, 4, 2, 3, 1, 5, 1, 2, 3, 4],
    'rating': [5.0, 4.0, 4.5, 4.5, 3.5, 4.0, 5.0, 4.0, 3.5, 4.0, 4.5, 4.0, 4.5, 3.0, 4.0,
               5.0, 4.5, 4.0, 4.5, 3.5, 4.0, 4.0, 4.5, 3.5, 5.0, 4.0, 4.5, 3.5]
})

# Movie metadata
movie_metadata = pd.DataFrame({
    'movieId': [1, 2, 3, 4, 5],
    'title': ['Movie Alpha', 'Movie Beta', 'Movie Gamma', 'Movie Delta', 'Movie Epsilon'],
    'genres': ['Action', 'Comedy', 'Drama', 'Thriller', 'Romance']
})

print("Sample Ratings Data:")
print(f"Total ratings: {len(raw_ratings)}")
print(f"Unique users: {raw_ratings['userId'].nunique()}")
print(f"Unique movies: {raw_ratings['movieId'].nunique()}")

In [ ]:
# Compute rating statistics from raw data
stats = compute_rating_statistics(
    raw_ratings,
    movie_id_column='movieId',
    rating_column='rating'
)

print("Rating Statistics per Movie:")
print(stats)

In [ ]:
# Merge movie metadata with rating statistics
movies_with_stats = merge_movie_with_stats(
    movie_metadata,
    stats,
    movie_id_column='movieId'
)

print("Movies with Rating Statistics:")
print(movies_with_stats)

In [ ]:
# Now get recommendations using the merged data
final_recommendations = get_top_n_recommendations(
    movies_with_stats,
    n=5,
    min_votes=3  # Lower threshold for small dataset
)

print("Final Top Recommendations:")
print(final_recommendations[['title', 'genres', 'avg_rating', 'rating_count', 'popularity_score']])

## 4.5 Baseline Model Evaluation

The weighted popularity baseline provides:

**Strengths:**
- Simple to implement and understand
- Fast computation (O(n) for n movies)
- No cold-start problem for new users
- Works well for popular content discovery

**Limitations:**
- Not personalized (same recommendations for everyone)
- Favors popular items (filter bubble risk)
- Ignores user preferences and item features

This baseline serves as a comparison point for more sophisticated models.

In [ ]:
# Compare effect of different weight factors
print("Effect of Weight Factor on Scores:")
print("="*60)

# Test with different weight factors
for wf in [0.25, 0.5, 1.0, 2.0]:
    print(f"\nWeight Factor = {wf}:")
    ranked = rank_movies_globally(
        movies_with_stats,
        min_votes=3,
        weight_factor=wf
    )
    for idx, row in ranked.head(3).iterrows():
        print(f"  {row['title']}: {row['popularity_score']:.3f}")

---

## Summary

This notebook covered Days 2-3 of ML exploration:

**Day 2 - Feature Engineering:**
1. **Feature Extraction Planning**: Defined approach for genre, tag, and temporal features
2. **Train/Test Split Design**: Implemented random and temporal split strategies
3. **Similarity Functions**: Created helpers for cosine, Jaccard, and Euclidean similarity

**Day 3 - Baseline Model:**
4. **Weighted Popularity Scoring**: Bayesian average for robust movie scoring
5. **Global Movie Ranking**: Ordering movies by popularity
6. **Top-N Recommendations**: Both global and personalized recommendation generation

### Next Steps

- [ ] Load actual MovieLens data and apply feature extraction
- [ ] Evaluate baseline model on real data
- [ ] Build content-based recommendation models
- [ ] Implement collaborative filtering approach
- [ ] Compare model performance against baseline

In [ ]:
# Summary of all helper functions available:

print("Feature Engineering Functions:")
print("  - encode_genres_onehot(): One-hot encode movie genres")
print("  - vectorize_tags_tfidf(): TF-IDF vectorize movie tags")
print("  - create_year_features(): Extract year and decade from titles")
print("  - FeaturePipeline: Complete feature extraction pipeline")
print("")
print("Split Functions:")
print("  - create_train_val_test_split(): Random train/val/test split")
print("  - create_temporal_split(): Time-based train/val/test split")
print("")
print("Similarity Functions:")
print("  - compute_cosine_similarity(): Cosine similarity for sparse features")
print("  - compute_jaccard_similarity(): Jaccard similarity for binary features")
print("  - compute_euclidean_similarity(): Euclidean distance-based similarity")
print("  - get_top_similar_items(): Get top-N similar items")
print("  - compute_weighted_similarity(): Combine multiple similarity matrices")
print("")
print("Baseline Model Functions:")
print("  - compute_weighted_popularity_score(): Single movie weighted score")
print("  - compute_popularity_scores(): Batch compute scores for all movies")
print("  - rank_movies_globally(): Rank movies by popularity score")
print("  - filter_by_minimum_votes(): Filter movies by vote threshold")
print("  - get_top_n_recommendations(): Get top-N global recommendations")
print("  - get_personalized_top_n(): Get personalized top-N (excluding watched)")
print("  - compute_rating_statistics(): Compute avg_rating and count from ratings")
print("  - merge_movie_with_stats(): Merge movie metadata with rating stats")